Q1: You are designing an online game and have two network layer protocols to choose from:
TCP and UDP. The online game is a chess simulator played entirely within browsers. You are in
charge of two features:

1. an implementation of real time voice chat,
2. the network handlers that send player data from the end user’s clients to your dedicated,
central servers

Which protocols do you use for each and why? You do not need to code these!

1. UDP is the better data transfer protocol choice for voice chat. Latency/lag is very important for the quality of voice calls, and a voice chat implementation in TCP would be both slower and prone to delays since the metadata that TCP packets are wrapped in would need to be parsed with each packet transfer and the state of each browser would need to be tracked. Additionally, small amounts of packet loss would not be too impactful for the overall audio quality of the call, so the downsides of UDP would not have a large effect on user experience.

2. TCP is the better data transfer protocol for handling player data.

Q2: Database dark arts:alchemy?
An advanced database module that some individuals use is called sqlalchemy, which you can
find here: https://www.sqlalchemy.org/. This module does something very special known as
ORM - object-relational mapping. Do a little research and then, in your own words, explain what
you think the unique, core purpose of this module is and why this feature is its distinguishing
property.

Object-relational mapping is a way of using object-oriented coding practices to query and interact with relational databases. By abstracting the database system, ORM libraries encourage good coding practices like code maintenance (because the data model is only implemented in one place, code is easier to maintain and update) and sanitized queries. Also, by leveraging the user's existing knowledge of Python and object-oriented programming practices, SQLAlchemy allows a user to implement and use SQL databases without requiring too much knowledge of SQL to get started.

Coding 1: Networked number guessing.
Using a network protocol of your choice, code a client/server Python program that implements a
number guessing game. You may have to implement this in two notebooks to test it!
The game should work in this fashion (4 points for each functionality piece):
1. The server is always run first.
2. The client connects, and when connected, asks the user for a numerical guess in a
range from 0 to 100.
3. The server should receive guesses, validate the guess, and send back either a high, low,
or correct message.
4. If the server sends back the correct message, the client should do something fun or
interesting to celebrate this fact - more than merely print a text string.
5. If the server sends back high or low, the client should then prompt for another guess.

In [1]:
import json
import random
import socket

def tcp_server(tcp_server_host, tcp_server_port) -> socket.socket:
    """
    make a socket object that will serve as the server

    :param tcp_server_host: designate the machine hosting the TCP server
    :param tcp_server_port: designate the port the host machine will listen through

    :return: a TCP socket object
    """

    # socket.SOCK_STREAM specifies that the socket should listen for TCP connections
    tcp_server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    # bind the socket and open the previously designated port for listening
    tcp_server_socket.bind((tcp_server_host, tcp_server_port))

    # argument passed to listen is the backlog size where the backlog is the maximum
    # number of pending TCP connections that the socket can queue at once. connections
    # over the maximum backlog size will be turned away/dropped
    tcp_server_socket.listen(5)

    return tcp_server_socket


def random_number_server(host="localhost", port=3360, packet_size=128):
    """
    start the server that will host the random number guessing game

    :param host: designate the machine hosting the TCP server (localhost by default)
    :param port: designate the port the host machine will listen through (3360 by default)
    :param packet_size: how large of a chunk of data the server should receive with any one data transmission (128 bytes by default)

    :return:
    """

    # instantiate the TCP socket object
    server = tcp_server(host, port)

    # choose a random integer between 0 and 100 inclusive
    random_number = random.choice(range(0,101))

    # print the random number for debugging purposes
    print(random_number)

    # listen for an attempted TCP connection and acknowledge on connect
    # to ensure that the client can handle a rejected connection correctly
    client_socket, address = server.accept()
    client_socket.send("connected".encode())

    # while the guess received is incorrect, maintain the connection
    # so that more guesses can be made
    correct_guess = False
    while not correct_guess:

        # get the guess from the client
        data = client_socket.recv(packet_size)

        # decode the guess from the client, and send a JSON as a response
        try:
            if random_number > int(data.decode()):
                result = {
                    "result": "too small",
                    "image": "https://media.tenor.com/hv9sKiPlpooAAAAC/gif-shinobi-you-are-wrong.gif"
                }
                client_socket.send(json.dumps(result).encode())
            elif random_number < int(data.decode()):
                result = {
                    "result": "too big",
                    "image": "https://media.tenor.com/hv9sKiPlpooAAAAC/gif-shinobi-you-are-wrong.gif"
                }
                client_socket.send(json.dumps(result).encode())
            else:
                result = {
                    "result": "just right",
                    "image": "https://media.tenor.com/qNPpRT04stcAAAAd/you-won-willy-wonka-and-the-chocolate-factory.gif"
                }
                client_socket.send(json.dumps(result).encode())

                # close the client-server connection if the guess is right to clean up
                correct_guess = True
                client_socket.close()

        # handle the case where the user input cannot be cast to an int
        except ValueError:
            result = {
                    "result": "wrong type",
                    "image": "https://media.tenor.com/hv9sKiPlpooAAAAC/gif-shinobi-you-are-wrong.gif"
                }
            client_socket.send(json.dumps(result).encode())

        # capture general exceptions for debugging purposes
        except Exception as e:
            print(f"Error type {type(e).__name__} with message {e}")

    # close the server socket for clean up
    server.close()

In [2]:
random_number_server(port=3360, packet_size=4096)

37


Coding 2: REQUEST? NO! I demand gracefully.
Using the requests module, request four different pieces of data from an API of your choosing.
You may use the one(s) in the class slides if you wish. We shall define different to mean that
each piece of data is unique in its content - no grabbing one piece of data with four different
filenames! (10 - 2.5 points per piece of data)
Once requested, your code should store this data locally on your device in an appropriate
persistent file format. You must use at least two different formats - I recommend JSON, XML, or
databases, though you can pickle if you wish. (2.5 points for each)

In [51]:
import requests

# server for accessing data on the human GRCh37 assembly
server = "http://grch37.rest.ensembl.org/"

# content header to let the API know how we would like our output
headers = {"Content-Type": "application/json"}

# our gene of interest is CYP2D6

In [3]:
import json
import requests

class HumanGeneDownload:
    """
    class to download the ensembl-associated sequences for a
    particular gene symbol (e.g. BRCA1)
    """

    # server for accessing data on the human GRCh37 assembly
    __ensembl_server = "http://grch37.rest.ensembl.org/"

    def __init__(self, gene_name: str):
        self.__gene_name = gene_name
        self.__ensembl_ids = self.__get_ensembl_ids()
        self.__make_gene_sequence_outputs()

    def gene_name(self):
        return self.__gene_name

    def ensembl_ids(self):
        return self.__ensembl_ids

    def __get_ensembl_ids(self):
        """
        get a list of ensembl ids associated with the gene of interest

        :return: a list of dicts containing the associated ensembl ids and the types of ids
        """

        # content header to let the API know how information is being sent
        headers = {"Content-Type": "application/json"}

        r = requests.get(
            self.__ensembl_server + f"xrefs/symbol/homo_sapiens/{self.__gene_name}",
            headers=headers,
        )

        if r.json():
            return r.json()
        else:
            raise ValueError("Invalid gene name. Expected a well-characterized human gene name")

    def __make_gene_sequence_outputs(self):
        """
        get the gene sequences for each ensembl gene id associated with the gene symbol and write
        to XML and JSON output formats

        :return: dict of gene ids and their sequences
        """

        # content header to let the API know to give a response as a json
        json_headers = {
            "Content-Type": "application/json",
            "Accept": "application/json"
        }

        # content header to let the API know to give a response as a xml
        xml_headers = {
            "Content-Type": "application/json",
            "Accept": "text/xml"
        }

        # get only the ids that are gene ids. associated ensembl ids can also be transcript ids which aren't
        # valid stable IDs for the sequence endpoint
        gene_ids = [ensembl_id["id"] for ensembl_id in self.__ensembl_ids if ensembl_id["type"] == "gene"]
        data = json.dumps({"ids": gene_ids})

        # first get a response in JSON format
        r = requests.post(
            self.__ensembl_server + f"sequence/id",
            headers=json_headers,
            data=data
        )
        json_response = r.json()

        # then get a response in XML format
        r = requests.post(
            self.__ensembl_server + f"sequence/id",
            headers=xml_headers,
            data=data
        )
        xml_response = r.content.decode()

        # write relevant output formats
        with open(f"{self.__gene_name}_ensembl_sequences.json", "w") as outfile:
            json.dump(json_response, outfile, indent=4)

        with open(f"{self.__gene_name}_ensembl_sequences.xml", "w") as outfile:
            outfile.write(xml_response)

In [4]:
HumanGeneDownload("brca1")
HumanGeneDownload("map3k15")
HumanGeneDownload("p53")
HumanGeneDownload("eln")